# Classifying 90+ Anime Characters with Transfer Learning


## Dataset Processing

First, install the prerequisite libraries:

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import os
import tensorflow as tf
import zipfile

Extract the dataset:

In [ ]:
base_dir = '/content/anime_dataset'

with zipfile.ZipFile("/content/drive/My Drive/dataset_cropped.zip") as datazip:
  datazip.extractall(base_dir)

train_dir = os.path.join(base_dir, 'cropped_images', 'train')
train_dir = os.path.join(base_dir, 'cropped_images', 'validation')

In [5]:
base_dir = '/content/anime_dataset'
train_dir = os.path.join(base_dir, 'cropped_images', 'train')
validation_dir = os.path.join(base_dir, 'cropped_images', 'validation')

Setup data augmentation/batch sizes for input:





In [29]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input, # Preprocess images for ResNet50
    rotation_range=30, # Randomly rotate the image up to 30 degrees
    horizontal_flip=True # Randomly flip the image horizontally
)

# No need for data augmentation in validation
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical', # Since we're classifying many different classes 
    batch_size=32, # Flow in batches of 32 images
    target_size=(244,244) # Reshape images to max image dimensions for ResNet50
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    class_mode='categorical',
    batch_size=32,
    target_size=(244,244)
)

Found 6341 images belonging to 94 classes.
Found 1063 images belonging to 94 classes.


Import and configure the base model (Inception V3):

In [31]:
base_model = tf.keras.applications.ResNet50(
    include_top=False, # Don't include last output layer for transfer learning
    input_shape=(244,244,3),
    weights='imagenet' # Import the weights for imagenet
)

base_model.trainable = False # Freeze the layers so we're not retraining

94773248/94765736 [==============================] - 1s 0us/step


## Model Building

Create our own layers:


In [32]:
# Pool the 3D output into a 2D feature vector to connect conv layers to our fully connected layers
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D() 
fc_layer = tf.keras.layers.Dense(1024, activation='relu')
output_layer = tf.keras.layers.Dense(94, activation='softmax') # Since we have 94 classes

Build our model:

In [33]:
inputs = tf.keras.Input(shape=(244,244,3))
x = base_model(inputs, training=False) # We need training=False since we have batch normalization layers in ResNet50
x = global_avg_layer(x)
x = tf.keras.layers.Dropout(0.2)(x) # For regularization
# x = fc_layer(x)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = output_layer(x)

model = tf.keras.Model(inputs, outputs)

Compile our model:

In [34]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 94)                192606    
Total params: 23,780,318
Trainable params: 192,606
Non-trainable params: 23,587,712
_________________________________________________________________


## Training

Train the model:


In [35]:
from PIL import PngImagePlugin
import h5py

LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

chkpoint_fp = '/content/drive/My Drive/chkpoints/resnetmodel.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=chkpoint_fp,
    monitor='val_accuracy',
    save_weights_only=True,
    save_best_only=True,
    mode='max'
)

In [ ]:
history = model.fit(
    x=train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[model_checkpoint],
    steps_per_epoch=198 # Since we have 6341 training images and batch sizes of 32, after 198 batches we'll have gone through a little less than the whole training set
)

Epoch 1/100
198/198 [==============================] - 165s 836ms/step - loss: 4.7432 - accuracy: 0.0274 - val_loss: 4.1996 - val_accuracy: 0.0677
Epoch 2/100
198/198 [==============================] - 169s 854ms/step - loss: 4.0896 - accuracy: 0.0802 - val_loss: 3.7186 - val_accuracy: 0.1543
Epoch 3/100
198/198 [==============================] - 169s 853ms/step - loss: 3.6284 - accuracy: 0.1667 - val_loss: 3.3513 - val_accuracy: 0.2408
Epoch 4/100
198/198 [==============================] - 168s 851ms/step - loss: 3.2486 - accuracy: 0.2360 - val_loss: 3.0751 - val_accuracy: 0.3029
Epoch 5/100
198/198 [==============================] - 167s 846ms/step - loss: 2.9621 - accuracy: 0.3010 - val_loss: 2.8511 - val_accuracy: 0.3603
Epoch 6/100
198/198 [==============================] - 168s 848ms/step - loss: 2.7435 - accuracy: 0.3484 - val_loss: 2.6830 - val_accuracy: 0.3932
Epoch 7/100
198/198 [==============================] - 168s 849ms/step - loss: 2.5459 - accuracy: 0.4083 - val_loss: 2

KeyboardInterrupt: ignored